In [11]:
import pandas as pd
import numpy as np

file='RM_Database.csv'
df=pd.read_csv(file)
print (df.columns)
cols=['mrn','treatment_end_date','Last Treatment Date','RM_date_','Date of Progression','last_date_on_file']
df=df[cols]
df=df.set_index('mrn')
# drop lines that have nothing at all
df.dropna(inplace=True,subset=['RM_date_','Date of Progression','last_date_on_file'],how='all')
#df.to_csv('examine.csv',index=True)
print (df.head())

Index(['mrn', 'pt_name', 'gender', 'race', 'ethnicity', 'date_of_birth',
       'date_of_death', 'drinks_per_week', 'TOBACCO_USER', 'pack_years',
       'Stage', 'TNM', 'T', 'N', 'M', 'narrative', 'Dx_', 'treatment_end_date',
       'RM_flag', 'RM_date_', 'last_date_on_file', 'date_of_primary_diagnosis',
       'primary_diagnoses', 'secondary_diagnoses', 'Last Treatment Date',
       'Date of Progression'],
      dtype='object')
       treatment_end_date  Last Treatment Date    RM_date_  \
mrn                                                          
765010                NaN  2007-06-19 00:00:00         NaN   
770827                NaN  2010-06-11 00:00:00         NaN   
788213                NaN  2010-04-14 00:00:00         NaN   
852008         2015-04-01           10/23/2015  2015-10-13   
995741                NaN  2007-10-24 00:00:00         NaN   

       Date of Progression last_date_on_file  
mrn                                           
765010          2007-10-03            

In [19]:
from math import isnan
### goal for chains is three columns: date, mrn, is_RM
### each mrn should have two entries
print ('starting number entires:',df.shape[0])
print (isnan(np.nan))

data=[]
for index,row in df.iterrows():
    mrn=index
    ### get start - prefer Last Treatment Date over treatment_end_date
    LTD=row['Last Treatment Date']
    TED=row['treatment_end_date']
    if not pd.isna(LTD):
        start=LTD
    else:
        start=TED
    ### get end - prefer Date of Progression over RM_date_, if DOP is nan, then use last date on file
    # for is_RM, set to 1 if you are using DOP or RM_date_,set to 0 if you use last date on file
    DOP=row['Date of Progression']
    RMD=row['RM_date_']
    if not pd.isna(DOP):
        end=DOP
        is_RM=1
    elif pd.isna(DOP) and not pd.isna(RMD):
        end=RMD
        is_RM=1
    elif pd.isna(DOP) and pd.isna(RMD):
        end=row['last_date_on_file']
        is_RM=0
    datum=[start,mrn,0]
    data.append(datum)
    datum=[end,mrn,is_RM]
    data.append(datum)
df_MCs=pd.DataFrame(data=data,columns=['date_of_service_dttm','mrn','is_RM'])
print(df_MCs)
df_MCs.to_csv('RM_MCs_DB.csv',index=False)

starting number entires: 228
True
    date_of_service_dttm      mrn  is_RM
0    2007-06-19 00:00:00   765010      0
1             2007-10-03   765010      1
2    2010-06-11 00:00:00   770827      0
3             2010-11-12   770827      1
4    2010-04-14 00:00:00   788213      0
..                   ...      ...    ...
451           2015-04-01  3506990      1
452  2015-08-31 00:00:00  3538715      0
453           2016-04-13  3538715      1
454  2013-09-19 00:00:00  3544750      0
455           2014-03-01  3544750      1

[456 rows x 3 columns]
